# Trabalho 1 - Nivelamento

Considere os datasets abaixo, estratégias de pré-processamento, medidas de avaliação, métodos de comparação estatística e os seguintes algoritmos de aprendizado de máquina: árvore de decisão, random forest e k-nearest neighbor. A partir disso, responda as seguintes perguntas:

1. Qual o algoritmo de AM mais adequado para cada problema?
2. Qual o algoritmo de AM mais adequado para todos os problemas?
knn:85.94066985255232
Arvore de decisão:85.42638695604664
Random Florest:86.62282691074151
Partindo de uma comparação direta entre os resultados do cross validation usando o msm conjunto de dados para todos, o algoritmo de random florest é o melhor, no entanto é o mais demorado também demorando 31 minutos para processar tudo, enquanto so demais demoraram apenas 1 minuto cada, em segundo fica o knn e terceiro a arvore de decisão que tem tempos similares. As únicas alterações feitas no dataset foi a seleção de atributos, e a conversão de atributos nominais para numéricos/binários utilizando o getdummies ou a conversão direta para os que tinham apenas duas classes.

Para responder essas questões construa um notebook no colab ou um ambiente similar. Documente de forma clara cada passo e justifique suas decisões.

In [1]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
import matplotlib.patches as mpatches
import matplotlib as plt
from scipy.io import arff
import urllib.request
import pandas as pd
import numpy as np
import io

In [2]:
'''Link com os datasets'''
PhishingWebsites_link = "https://www.openml.org/data/download/1798106/phpV5QYya"
arrhythmia_link       = "https://www.openml.org/data/download/53551/arrhythmia.arff"
Satellite_link        = "https://www.openml.org/data/download/16787463/phpZrCzJR"
airlines_link         = "https://www.openml.org/data/download/66526/phpvcoG8S"
AedesSex_link         = "https://github.com/denismr/Classification-and-Counting-with-Recurrent-Contexts/raw/master/codeAndData/data/AedesSex.csv"
phoneme_link          = "https://www.openml.org/data/download/1592281/php8Mz7BG"
adult_link            = "https://www.openml.org/data/download/1595261/phpMawTba"

In [3]:
def loadDataset_Arff(url: str) -> pd.DataFrame:
  '''Função responsável por ler os datasets do tipo .arff'''
  ftpstream = urllib.request.urlopen(url)
  return pd.DataFrame(arff.loadarff(io.StringIO(ftpstream.read().decode('utf-8')))[0])
  
def get_dummies(df: pd.DataFrame, col:str) -> pd.DataFrame:
  '''Função responsável por separar os dados nominais em outras colunas.'''
  return pd.get_dummies(df,prefix=col,prefix_sep='.',columns=[col]).copy()

In [4]:
def proportion(Dataset,target="Target"):
  '''Função que gera um gráfico de proporção dos valores targets'''
  target_count = Dataset[target].value_counts()
  print('Class 0:', target_count[0])
  print('Class 1:', target_count[1])
  target_count.plot(kind='bar', title='Count (target)');
  
def normalize(dataset,target='Target') -> pd.DataFrame:
  '''Função que retorna o dataset normalizado'''
  X = dataset.drop([target], axis=1).copy()
  Y = dataset[target].copy()

  scaler = MinMaxScaler()
  X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
  dataset = X
  dataset[target] = Y
  return dataset

In [5]:
def knn(dataset,n_neighbors=3,target='Target'):
  '''Função que aplica o knn'''
  X_train, X_test, y_train, y_test = train_test_split(dataset.drop([target], axis=1), dataset[target], test_size=0.3)
  model = KNeighborsClassifier(n_neighbors=3)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print(accuracy_score(y_test, y_pred)*100)

def treeDecision(dataset,target='Target'):
  '''Função que aplica a arvore de decisão'''
  X_train, X_test, y_train, y_test = train_test_split(dataset.drop([target], axis=1), dataset[target], test_size=0.3)
  model = DecisionTreeClassifier()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print(accuracy_score(y_test, y_pred)*100)

def RForestDecision(dataset,target='Target'):
  '''Função que aplica random florest'''
  X_train, X_test, y_train, y_test = train_test_split(dataset.drop([target], axis=1), dataset[target], test_size=0.3)
  model = RandomForestClassifier()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print(accuracy_score(y_test, y_pred)*100)
  

In [6]:
def TTS_knn(dataset, x ,y ,n_neighbors=3):
  '''Função que aplica o knn com x e y previamente setados'''
  model = KNeighborsClassifier(n_neighbors=3)
  return cross_val_score(model, x, y, cv=20, scoring="accuracy")

def TTS_treeDecision(dataset, x ,y):
  '''Função que aplica a arvore de decisão com x e y previamente setados'''
  model = DecisionTreeClassifier()
  return cross_val_score(model, x, y, cv=20, scoring="accuracy")

def TTS_RForestDecision(dataset, x ,y):
  '''Função que aplica o random florest com x e y previamente setados'''
  model = RandomForestClassifier()
  return cross_val_score(model, x, y, cv=20, scoring="accuracy")

In [7]:
def selectAttributes(dataset):
  '''Função que aplica o knn com x e y previamente setados'''
  model = DecisionTreeClassifier()#max_leaf_nodes=10
  feature_ = RFE(model,n_features_to_select=5, step=1)
  return feature_.fit(dataset.drop(['Target'],axis=1),dataset['Target']).support_

def removeColumns(dataset):
  '''Função que faz a seleção de atributos e retorna uma lista com o nome das colunas que serão removidas'''
  attributes = selectAttributes(dataset)
  dropListIndex = np.where(attributes == False)[0]
  dropList = []
  for index in dropListIndex:
    if(dataset.columns[index] != "Target"):
      dropList.append(dataset.columns[index])
  return dropList


# Lendo todos os datasets

In [8]:
InitDataset_PhishingWebsites = loadDataset_Arff(PhishingWebsites_link)
InitDataset_arrhythmia = loadDataset_Arff(arrhythmia_link)
InitDataset_Satellite = loadDataset_Arff(Satellite_link)
InitDataset_airlines = loadDataset_Arff(airlines_link)
InitDataset_AedesSex = pd.read_csv(AedesSex_link)
InitDataset_phoneme = loadDataset_Arff(phoneme_link)
InitDataset_adult = loadDataset_Arff(adult_link)

Abaixo será feito a conversão das classes targets para True e False, e trocado o nome da coluna para Target para as que não estão. Também será feito uma analise de distribuição e procura de dados nominais.

In [9]:
Dataset_PhishingWebsites = InitDataset_PhishingWebsites.copy()
Dataset_PhishingWebsites['Target'] = [True if x == '-1'.encode() else False for x in Dataset_PhishingWebsites['Result']]
Dataset_PhishingWebsites = Dataset_PhishingWebsites.drop(['Result'],axis=1)
Dataset_PhishingWebsites = Dataset_PhishingWebsites.drop(removeColumns(Dataset_PhishingWebsites),axis=1)

In [ ]:
Dataset_arrhythmia = InitDataset_arrhythmia.copy()
Dataset_arrhythmia['Target'] = [True if x == 'P'.encode() else False for x in Dataset_arrhythmia['binaryClass']]
Dataset_arrhythmia = Dataset_arrhythmia.drop(['binaryClass','J'],axis=1) #Remoção da coluna J por ter 98% dos dados NaN
# Dataset_arrhythmia = Dataset_arrhythmia.drop(removeColumns(Dataset_arrhythmia),axis=1) 
# Dataset_arrhythmia = Dataset_arrhythmia.dropna() #Remoção das linhas com valores faltantes

In [10]:
##Sem nominais usando a função select_nominals
Dataset_Satellite = InitDataset_Satellite.copy()
Dataset_Satellite['Target'] = [True if x == 'Anomaly'.encode() else False for x in Dataset_Satellite['Target']]
Dataset_Satellite = Dataset_Satellite.drop(removeColumns(Dataset_Satellite),axis=1) 

In [11]:
##Dataset com muitos dados nominais, para aplicar o knn será necessário usar a função getdummies em AirportFrom e AirportTo.
Dataset_airlines = InitDataset_airlines.copy()
Dataset_airlines['Target'] = [True if x == '1'.encode() else False for x in Dataset_airlines['Delay']]
Dataset_airlines = Dataset_airlines.drop(['Delay'],axis=1)

Dataset_airlines['Airline'] = Dataset_airlines['Airline'].astype('category')
Dataset_airlines['AirportFrom'] = Dataset_airlines['AirportFrom'].astype('category')
Dataset_airlines['AirportTo'] = Dataset_airlines['AirportTo'].astype('category')

Dataset_airlines['Airline'] = Dataset_airlines['Airline'].cat.codes
Dataset_airlines['AirportFrom'] = Dataset_airlines['AirportFrom'].cat.codes
Dataset_airlines['AirportTo'] = Dataset_airlines['AirportTo'].cat.codes

Dataset_airlines = Dataset_airlines.drop(removeColumns(Dataset_airlines),axis=1) 

In [12]:
Dataset_AedesSex = InitDataset_AedesSex.copy()
Dataset_AedesSex['Target'] = [True if x == 'F' else False for x in Dataset_AedesSex['sex']]
Dataset_AedesSex = Dataset_AedesSex.drop(['sex'],axis=1)
Dataset_AedesSex = Dataset_AedesSex.drop(removeColumns(Dataset_AedesSex),axis=1) 

In [13]:
##Dataset desbalanceado, analisar como se comporta nos algoritmos de aprendizagem para saber se é necessário balancear.
Dataset_phoneme = InitDataset_phoneme.copy()
Dataset_phoneme['Target'] = [True if x == '1'.encode() else False for x in Dataset_phoneme['Class']]
Dataset_phoneme = Dataset_phoneme.drop(['Class'],axis=1)
Dataset_phoneme = Dataset_phoneme.drop(removeColumns(Dataset_phoneme),axis=1)

In [ ]:
# ##Dataset com muitas colunas nominais, será necessário gerar o dummie das mesmas.
Dataset_adult = InitDataset_adult.copy()
Dataset_adult['Target'] = [True if x == '<=50K'.encode() else False for x in Dataset_adult['class']]
Dataset_adult = Dataset_adult.drop(['class'],axis=1)
Dataset_adult['workclass'] = [x.decode('UTF-8') for x in Dataset_adult['workclass']]
Dataset_adult['education'] = [x.decode('UTF-8') for x in Dataset_adult['education']]
Dataset_adult['marital-status'] = [x.decode('UTF-8') for x in Dataset_adult['marital-status']]
Dataset_adult['occupation'] = [x.decode('UTF-8') for x in Dataset_adult['occupation']]
Dataset_adult['relationship'] = [x.decode('UTF-8') for x in Dataset_adult['relationship']]
Dataset_adult['race'] = [x.decode('UTF-8') for x in Dataset_adult['race']]
Dataset_adult['sex'] = [x.decode('UTF-8') for x in Dataset_adult['sex']]
Dataset_adult['native-country'] = [x.decode('UTF-8') for x in Dataset_adult['native-country']]

Dataset_adult = Dataset_adult.dropna() #Remoção das linhas com valores faltantes
Dataset_adult = get_dummies(Dataset_adult,'workclass')
Dataset_adult = get_dummies(Dataset_adult,'education')
Dataset_adult = get_dummies(Dataset_adult,'marital-status')
Dataset_adult = get_dummies(Dataset_adult,'occupation')
Dataset_adult = get_dummies(Dataset_adult,'relationship')
Dataset_adult = get_dummies(Dataset_adult,'race')
Dataset_adult = get_dummies(Dataset_adult,'sex')
Dataset_adult = get_dummies(Dataset_adult,'native-country')
Dataset_adult = Dataset_adult.drop(removeColumns(Dataset_adult),axis=1)

In [15]:
##Normalizando todos os dados
NOR_Dataset_PhishingWebsites = normalize(Dataset_PhishingWebsites)
# NOR_Dataset_arrhythmia = normalize(Dataset_arrhythmia)
NOR_Dataset_Satellite = normalize(Dataset_Satellite) #DESBALANCEADO
NOR_Dataset_AedesSex = normalize(Dataset_AedesSex)
NOR_Dataset_phoneme = normalize(Dataset_phoneme)
NOR_Dataset_adult = normalize(Dataset_adult)
NOR_Dataset_airlines = normalize(Dataset_airlines) # Dataset estourando a memoria.

In [ ]:
knn(NOR_Dataset_PhishingWebsites) #93.8498643352427
# knn(NOR_Dataset_arrhythmia) 
knn(NOR_Dataset_Satellite) #99.2156862745098
knn(NOR_Dataset_AedesSex) #98.73611111111111
knn(NOR_Dataset_phoneme) #88.03945745992601
knn(NOR_Dataset_adult) #81.71022998703337
knn(NOR_Dataset_airlines) #81.71022998703337


treeDecision(NOR_Dataset_PhishingWebsites)#96.02050045221586
# treeDecision(NOR_Dataset_arrhythmia)
treeDecision(NOR_Dataset_Satellite)#99.08496732026144
treeDecision(NOR_Dataset_AedesSex)#98.19444444444444
treeDecision(NOR_Dataset_phoneme)#87.05302096177559
treeDecision(NOR_Dataset_adult)#82.01733433426602
treeDecision(NOR_Dataset_airlines)


RForestDecision(NOR_Dataset_PhishingWebsites)#96.80434127223396
# RForestDecision(NOR_Dataset_arrhythmia)
RForestDecision(NOR_Dataset_Satellite)#99.281045751634
RForestDecision(NOR_Dataset_AedesSex)#98.38888888888889
RForestDecision(NOR_Dataset_phoneme)#90.9987669543773
RForestDecision(NOR_Dataset_adult)#85.68893741895857
RForestDecision(NOR_Dataset_airlines)



In [17]:
##Separando os atributos dos datasets para testar qual o melhor algoritmo
NOR_Dataset_PhishingWebsites_y = NOR_Dataset_PhishingWebsites['Target']
NOR_Dataset_PhishingWebsites_x = NOR_Dataset_PhishingWebsites.drop(['Target'],axis=1)

NOR_Dataset_Satellite_y = NOR_Dataset_Satellite['Target']
NOR_Dataset_Satellite_x = NOR_Dataset_Satellite.drop(['Target'],axis=1)

NOR_Dataset_AedesSex_y = NOR_Dataset_AedesSex['Target']
NOR_Dataset_AedesSex_x = NOR_Dataset_AedesSex.drop(['Target'],axis=1)

NOR_Dataset_phoneme_y = NOR_Dataset_phoneme['Target']
NOR_Dataset_phoneme_x = NOR_Dataset_phoneme.drop(['Target'],axis=1)

NOR_Dataset_adult_y = NOR_Dataset_adult['Target']
NOR_Dataset_adult_x = NOR_Dataset_adult.drop(['Target'],axis=1)

NOR_Dataset_airlines_y = NOR_Dataset_airlines['Target']
NOR_Dataset_airlines_x = NOR_Dataset_airlines.drop(['Target'],axis=1)

In [18]:
##Calculando a media de todos os datasets a partir do cross validation juntamente com o knn
precisionListKnn = np.array([])
precisionListKnn = np.append(precisionListKnn,TTS_knn(NOR_Dataset_PhishingWebsites,NOR_Dataset_PhishingWebsites_x,NOR_Dataset_PhishingWebsites_y) ) 
precisionListKnn = np.append(precisionListKnn,TTS_knn(NOR_Dataset_Satellite,NOR_Dataset_Satellite_x,NOR_Dataset_Satellite_y) ) 
precisionListKnn = np.append(precisionListKnn,TTS_knn(NOR_Dataset_AedesSex,NOR_Dataset_AedesSex_x,NOR_Dataset_AedesSex_y) ) 
precisionListKnn = np.append(precisionListKnn,TTS_knn(NOR_Dataset_phoneme,NOR_Dataset_phoneme_x,NOR_Dataset_phoneme_y) ) 
precisionListKnn = np.append(precisionListKnn,TTS_knn(NOR_Dataset_adult,NOR_Dataset_adult_x,NOR_Dataset_adult_y) ) 
precisionListKnn = np.append(precisionListKnn,TTS_knn(NOR_Dataset_airlines,NOR_Dataset_airlines_x,NOR_Dataset_airlines_y) ) 
precisionListKnn.mean()*100# 85.94066985255232



85.94066985255232

In [19]:
##Calculando a media de todos os datasets a partir do cross validation juntamente com a árvore de decisão
precisionListTree = np.array([])
precisionListTree = np.append(precisionListTree,TTS_treeDecision(NOR_Dataset_PhishingWebsites,NOR_Dataset_PhishingWebsites_x,NOR_Dataset_PhishingWebsites_y) ) 
precisionListTree = np.append(precisionListTree,TTS_treeDecision(NOR_Dataset_Satellite,NOR_Dataset_Satellite_x,NOR_Dataset_Satellite_y) ) 
precisionListTree = np.append(precisionListTree,TTS_treeDecision(NOR_Dataset_AedesSex,NOR_Dataset_AedesSex_x,NOR_Dataset_AedesSex_y) ) 
precisionListTree = np.append(precisionListTree,TTS_treeDecision(NOR_Dataset_phoneme,NOR_Dataset_phoneme_x,NOR_Dataset_phoneme_y) ) 
precisionListTree = np.append(precisionListTree,TTS_treeDecision(NOR_Dataset_adult,NOR_Dataset_adult_x,NOR_Dataset_adult_y) ) 
precisionListTree = np.append(precisionListTree,TTS_treeDecision(NOR_Dataset_airlines,NOR_Dataset_airlines_x,NOR_Dataset_airlines_y) ) 
precisionListTree.mean()*100#85.42638695604664


85.42638695604664

In [20]:
##Calculando a media de todos os datasets a partir do cross validation juntamente com o algoritmo de random florest.
precisionListRf = np.array([])
precisionListRf = np.append(precisionListRf,TTS_RForestDecision(NOR_Dataset_PhishingWebsites,NOR_Dataset_PhishingWebsites_x,NOR_Dataset_PhishingWebsites_y) ) 
precisionListRf = np.append(precisionListRf,TTS_RForestDecision(NOR_Dataset_Satellite,NOR_Dataset_Satellite_x,NOR_Dataset_Satellite_y) ) 
precisionListRf = np.append(precisionListRf,TTS_RForestDecision(NOR_Dataset_AedesSex,NOR_Dataset_AedesSex_x,NOR_Dataset_AedesSex_y) ) 
precisionListRf = np.append(precisionListRf,TTS_RForestDecision(NOR_Dataset_phoneme,NOR_Dataset_phoneme_x,NOR_Dataset_phoneme_y) ) 
precisionListRf = np.append(precisionListRf,TTS_RForestDecision(NOR_Dataset_adult,NOR_Dataset_adult_x,NOR_Dataset_adult_y) ) 
precisionListRf = np.append(precisionListRf,TTS_RForestDecision(NOR_Dataset_airlines,NOR_Dataset_airlines_x,NOR_Dataset_airlines_y) ) 
precisionListRf.mean()*100#86.62282691074151

86.62282691074151

In [ ]:
precisionListRf 
# array([0.90958409, 0.93128391, 0.920434  , 0.92585895, 0.93670886,
#        0.90958409, 0.93851718, 0.91500904, 0.90958409, 0.93128391,
#        0.93128391, 0.94936709, 0.920434  , 0.93490054, 0.92947559,
#        0.86594203, 0.93297101, 0.91485507, 0.92934783, 0.92934783,
#        0.99607843, 0.99607843, 0.99215686, 0.98823529, 1.        ,
#        0.99215686, 0.99607843, 0.99607843, 0.99607843, 0.98823529,
#        1.        , 0.99215686, 0.98823529, 0.99215686, 0.99607843,
#        0.99607843, 0.99607843, 0.99215686, 0.99607843, 0.99607843,
#        0.97416667, 0.95166667, 0.97666667, 0.99083333, 0.99      ,
#        0.9925    , 0.98833333, 0.99583333, 0.99583333, 0.99333333,
#        0.95833333, 0.98166667, 0.9875    , 0.98833333, 0.99083333,
#        0.99083333, 0.9875    , 0.98833333, 0.99333333, 0.99583333,
#        0.93726937, 0.92619926, 0.91143911, 0.88560886, 0.92222222,
#        0.9       , 0.94074074, 0.92222222, 0.91111111, 0.89259259,
#        0.91481481, 0.91111111, 0.8962963 , 0.91111111, 0.92962963,
#        0.92962963, 0.91481481, 0.91851852, 0.9       , 0.8962963 ,
#        0.77568563, 0.76708964, 0.77682228, 0.77354627, 0.76863227,
#        0.78132678, 0.77477477, 0.77723178, 0.78705979, 0.77559378,
#        0.76863227, 0.76044226, 0.77190827, 0.76822277, 0.78419328,
#        0.77149877, 0.79238329, 0.77190827, 0.77927928, 0.77149877,
#        0.64542084, 0.60044494, 0.65324435, 0.60821684, 0.54681301,
#        0.54788832, 0.6061033 , 0.6368423 , 0.58237235, 0.58596908,
#        0.62660833, 0.63009381, 0.62304869, 0.65200786, 0.6024695 ,
#        0.5528199 , 0.5813712 , 0.596648  , 0.600838  , 0.63509956])

In [ ]:
precisionListTree
# array([0.90958409, 0.93128391, 0.91862568, 0.92585895, 0.93670886,
#        0.90958409, 0.93851718, 0.91500904, 0.90958409, 0.93128391,
#        0.93128391, 0.94936709, 0.920434  , 0.93490054, 0.92947559,
#        0.87137681, 0.93297101, 0.91485507, 0.92934783, 0.92934783,
#        0.99607843, 0.99215686, 0.98823529, 0.98823529, 1.        ,
#        0.98039216, 0.99215686, 0.99607843, 0.99215686, 0.98823529,
#        1.        , 0.99215686, 0.98431373, 0.98431373, 0.99607843,
#        0.99607843, 0.99215686, 0.99215686, 0.99215686, 0.96862745,
#        0.95333333, 0.935     , 0.9675    , 0.98166667, 0.9825    ,
#        0.98666667, 0.9775    , 0.9925    , 0.99416667, 0.99166667,
#        0.93333333, 0.975     , 0.97666667, 0.98416667, 0.98583333,
#        0.9825    , 0.98083333, 0.98583333, 0.99083333, 0.99416667,
#        0.89298893, 0.87084871, 0.8597786 , 0.84132841, 0.88148148,
#        0.87407407, 0.90740741, 0.86296296, 0.85925926, 0.85555556,
#        0.87777778, 0.85555556, 0.87777778, 0.87777778, 0.87777778,
#        0.88518519, 0.86296296, 0.8962963 , 0.88518519, 0.85555556,
#        0.75399099, 0.76299632, 0.75102375, 0.76167076, 0.74201474,
#        0.75593776, 0.76126126, 0.75798526, 0.75839476, 0.76617527,
#        0.75921376, 0.74774775, 0.75716626, 0.75552826, 0.75143325,
#        0.74365274, 0.76494676, 0.76044226, 0.76412776, 0.75266175,
#        0.62832777, 0.62977382, 0.65146459, 0.58456005, 0.52267418,
#        0.5988357 , 0.63869628, 0.59201305, 0.53090585, 0.55916052,
#        0.67559049, 0.66617227, 0.60458304, 0.64414698, 0.59616597,
#        0.59523898, 0.57584634, 0.58055545, 0.58945456, 0.62875895])

In [ ]:
precisionListKnn
# array([0.90596745, 0.92766727, 0.93490054, 0.92585895, 0.93851718,
#        0.92947559, 0.94575045, 0.91139241, 0.92766727, 0.92947559,
#        0.9403255 , 0.95298373, 0.91681736, 0.93670886, 0.93128391,
#        0.86050725, 0.89492754, 0.88768116, 0.90036232, 0.89492754,
#        0.99215686, 1.        , 0.99215686, 0.98823529, 0.99607843,
#        0.99215686, 0.99607843, 0.99215686, 0.98823529, 0.98823529,
#        0.99607843, 0.99607843, 0.98823529, 0.99215686, 0.99607843,
#        0.99607843, 0.99607843, 0.99215686, 0.99607843, 0.99215686,
#        0.97      , 0.94416667, 0.985     , 0.98916667, 0.98166667,
#        0.98333333, 0.9875    , 0.99416667, 0.99166667, 0.98833333,
#        0.95416667, 0.9825    , 0.9875    , 0.98583333, 0.99      ,
#        0.98333333, 0.9875    , 0.9875    , 0.99083333, 0.99333333,
#        0.90774908, 0.89298893, 0.87453875, 0.87084871, 0.91111111,
#        0.86666667, 0.91481481, 0.87407407, 0.8962963 , 0.85555556,
#        0.85925926, 0.91111111, 0.88148148, 0.91111111, 0.91851852,
#        0.87407407, 0.91481481, 0.89259259, 0.88148148, 0.9       ,
#        0.78141629, 0.76217765, 0.77846028, 0.77395577, 0.77436527,
#        0.77108927, 0.77600328, 0.75962326, 0.78869779, 0.79197379,
#        0.77354627, 0.76658477, 0.77067977, 0.76986077, 0.77723178,
#        0.77559378, 0.78583129, 0.77927928, 0.78746929, 0.76863227,
#        0.64074898, 0.59610679, 0.61690768, 0.60699321, 0.54377248,
#        0.53932293, 0.60413808, 0.63506248, 0.56056954, 0.54788832,
#        0.60361897, 0.63561867, 0.58704438, 0.62089807, 0.61440914,
#        0.5632022 , 0.58430049, 0.54862991, 0.60369313, 0.63695354])